In [1]:
import pickle

import pandas as pd
import dask.dataframe as dd

from py.consts import DATA_DIR

In [2]:
tomorrow_options = dd.read_parquet(DATA_DIR / f"prediction_preset.parquet")

with open(DATA_DIR / "model.pkl", "rb") as f:
    clf_model = pickle.load(f)

In [3]:
trained_roads = [col for col in clf_model.feature_names_in_ if col.startswith("wegnummer")]
tomorrow_options_roads = [col for col in tomorrow_options.columns if col.startswith("wegnummer")]
untrained_roads = [r for r in tomorrow_options_roads if r not in trained_roads]

print(untrained_roads)

tomorrow_options = tomorrow_options.drop(untrained_roads, axis=1)

tomorrow_options.compute()


untrained_cols = [col for col in list(tomorrow_options.columns) if col not in list(clf_model.feature_names_in_)]
missing_features = [f for f in list(clf_model.feature_names_in_) if f not in list(tomorrow_options.columns)]


from pprint import pprint
print()
print()
pprint(list(tomorrow_options.columns))
print(len(tomorrow_options.columns))
print()
pprint(list(clf_model.feature_names_in_))
pprint(len(clf_model.feature_names_in_))
print()
pprint(f"{len(untrained_cols) =} {untrained_cols = }")
print()
pprint(f"{len(missing_features) =} {missing_features = }")
print()

['wegnummer_N287', 'wegnummer_N368', 'wegnummer_N382', 'wegnummer_N410', 'wegnummer_N418', 'wegnummer_N467', 'wegnummer_N523', 'wegnummer_N571', 'wegnummer_N606', 'wegnummer_N621', 'wegnummer_N629', 'wegnummer_N667', 'wegnummer_N671', 'wegnummer_N675', 'wegnummer_N676', 'wegnummer_N680', 'wegnummer_N817', 'wegnummer_N927', 'wegnummer_N928']


['zijde_links',
 'zijde_rechts',
 'wegnummer_A10',
 'wegnummer_A12',
 'wegnummer_A13',
 'wegnummer_A15',
 'wegnummer_A16',
 'wegnummer_A17',
 'wegnummer_A18',
 'wegnummer_A2',
 'wegnummer_A20',
 'wegnummer_A200',
 'wegnummer_A208',
 'wegnummer_A22',
 'wegnummer_A256',
 'wegnummer_A27',
 'wegnummer_A270',
 'wegnummer_A28',
 'wegnummer_A29',
 'wegnummer_A30',
 'wegnummer_A31',
 'wegnummer_A32',
 'wegnummer_A325',
 'wegnummer_A326',
 'wegnummer_A348',
 'wegnummer_A35',
 'wegnummer_A37',
 'wegnummer_A4',
 'wegnummer_A44',
 'wegnummer_A5',
 'wegnummer_A50',
 'wegnummer_A58',
 'wegnummer_A59',
 'wegnummer_A6',
 'wegnummer_A65',
 'wegnummer_A67',
 'wegnu

In [4]:
tomorrow_options = tomorrow_options.drop(untrained_cols, axis=1)

for f in missing_features:
    tomorrow_options[f] = 0

In [5]:
tomorrow_options = tomorrow_options[list(clf_model.feature_names_in_)]
tomorrow_options.compute()

,zijde_links,zijde_rechts,month_August,month_December,month_February,month_January,month_July,month_June,month_March,month_May,...,stop_hour_14,stop_hour_15,stop_hour_16,stop_hour_17,stop_hour_18,stop_hour_19,stop_hour_20,stop_hour_21,stop_hour_22,stop_hour_23
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483547,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
483548,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
483549,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
483550,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [6]:
y = clf_model.predict(tomorrow_options)

print(y)
print(len(y))

[0 0 0 ... 0 0 0]
483552


In [7]:
# y_col = dd.from_array(y)
# tomorrow_options["y"] = y_col

from dask import delayed

# Define a delayed function to add the new column
@delayed
def add_new_column(df, new_column_data):
    df['y'] = new_column_data
    return df

tomorrow_options = add_new_column(tomorrow_options, y)

result = tomorrow_options.compute()
print(result.shape)

tomorrow_options = tomorrow_options[tomorrow_options.y == 1]

result = tomorrow_options.compute()
print(result.shape)
print(result.head())

(483552, 630)
(0, 630)
Empty DataFrame
Columns: [zijde_links, zijde_rechts, month_August, month_December, month_February, month_January, month_July, month_June, month_March, month_May, month_November, month_October, month_September, day_Monday, day_Saturday, day_Sunday, day_Thursday, day_Tuesday, day_Wednesday, wegnummer_A10, wegnummer_A12, wegnummer_A13, wegnummer_A15, wegnummer_A16, wegnummer_A17, wegnummer_A18, wegnummer_A2, wegnummer_A20, wegnummer_A200, wegnummer_A208, wegnummer_A22, wegnummer_A256, wegnummer_A27, wegnummer_A270, wegnummer_A28, wegnummer_A29, wegnummer_A30, wegnummer_A31, wegnummer_A32, wegnummer_A325, wegnummer_A326, wegnummer_A348, wegnummer_A35, wegnummer_A37, wegnummer_A4, wegnummer_A44, wegnummer_A5, wegnummer_A50, wegnummer_A58, wegnummer_A59, wegnummer_A6, wegnummer_A65, wegnummer_A67, wegnummer_A7, wegnummer_A73, wegnummer_A74, wegnummer_A76, wegnummer_A77, wegnummer_A79, wegnummer_A8, wegnummer_A9, wegnummer_N11, wegnummer_N14, wegnummer_N15, wegnummer_N1

In [8]:
reversed_df = result.drop("y", axis=1)
positives = pd.from_dummies(reversed_df, sep="_", default_category={"zijde": "beide", "wegnummer": "x", "month": "month_x", "day": "day_x", "start_hour": -1, "stop_hour": -1})

positives.head()

,zijde,month,day,wegnummer,start,stop


In [39]:
# this data is redundant by now
positives = positives.drop(["month", "day"], axis=1)

In [40]:
# TODO: remove once better separator is set
positives.start = positives.start.str.replace("hour_", "")
positives.stop = positives.stop.str.replace("hour_", "")

positives = positives.astype({"start": int, "stop": int})

positives.head()

,zijde,wegnummer,start,stop
139,links,N321,6,23
156,links,N321,8,9
157,links,N321,8,10
158,links,N321,8,11
159,links,N321,8,12


In [41]:
def get_max_and_min(group):
    group.start = group.start.min()
    group.stop = group.stop.max()
    return group

grouped = positives.groupby(["zijde", "wegnummer"])

x = grouped.apply(get_max_and_min).drop_duplicates()
x.reset_index(drop=True, inplace=True)

print(x)

       zijde wegnummer  start  stop
0      beide       A10      6    23
1      beide       A12      6    23
2      beide       A13      6    23
3      beide       A15      6    23
4      beide       A16      6    23
...      ...       ...    ...   ...
1690  rechts      N996      6    23
1691  rechts      N997      6    23
1692  rechts      N999      6    23
1693  rechts      Y351      6    23
1694  rechts         x      6    23

[1695 rows x 4 columns]


In [42]:
from py.consts import TOMORROW


filename = f"prediction_for_{TOMORROW.strftime('%Y-%m-%d')}.csv"
x.to_csv(DATA_DIR / filename)